# 主要功能：

- 自動化資料擷取
- 標準化資料處理
- 自動統計分析
- 自動生成報告
- 記錄檔管理
- 錯誤處理

# **1. 定期下載與更新 CSV 資料**
使用 `requests` 下載 CSV 檔案：

In [1]:
import pandas as pd
import requests

# API 連結
API_URL = "https://info.nhi.gov.tw/api/iode0000s01/Dataset?rId=A21030000I-L50007-001"
LOCAL_CSV_FILE = "antidepressant_usage.csv"

# 方式 1：直接從 API 讀取 CSV
def read_from_api():
    df = pd.read_csv(API_URL, encoding="utf-8")
    print("📡 已從 API 讀取最新資料")
    return df

# 方式 2：下載 CSV 並存檔，再讀取
def download_and_read():
    response = requests.get(API_URL)
    with open(LOCAL_CSV_FILE, "wb") as file:
        file.write(response.content)
    print(f"💾 已下載並存為 {LOCAL_CSV_FILE}")
    
    df = pd.read_csv(LOCAL_CSV_FILE, encoding="utf-8")
    return df

# 測試兩種方法
df_api = read_from_api()
print(df_api.head())  # 顯示 API 讀取的資料

df_local = download_and_read()
print(df_local.head())  # 顯示本地存檔後的資料


📡 已從 API 讀取最新資料
   年別  抗憂鬱藥物使用人數    性別＿男    性別＿女  年齡別＿30歲以下  年齡別＿31－40歲  年齡別＿41－50歲  \
0  94     791224  338709  452515     124063      114808      149683   
1  95     791932  337890  454042     113421      111495      150628   
2  96     812279  342728  469551     109130      113153      154526   
3  97     846317  356255  490062     108665      117088      161430   
4  98     874930  366268  508662     106507      119162      166298   

   年齡別＿51－65歲  年齡別＿65歲以上   臺北業務組  ...  中區業務組＿南投縣  南區業務組＿臺南市  南區業務組＿嘉義市  \
0      194363     208307  253436  ...          …          …          …   
1      200548     215840  259669  ...          …          …          …   
2      211550     223920  266505  ...          …          …          …   
3      224669     234465  278453  ...          …          …          …   
4      238197     244766  289098  ...          …          …          …   

   南區業務組＿雲林縣  南區業務組＿嘉義縣 高屏業務組＿高雄市 高屏業務組＿屏東縣 高屏業務組＿澎湖縣 東區業務組＿花蓮縣 東區業務組＿臺東縣  
0          …          …         …   


---

## **2. 資料清理與轉換**
讀取 CSV 並處理遺漏值、轉換格式等：

```python
# 讀取 CSV
df = pd.read_csv(csv_file, encoding="utf-8")

# 確認資料格式
print(df.head())

# 檢查缺失值
print(df.isnull().sum())

# 填補遺漏值（這裡示範填充 0，你可以根據需求調整）
df.fillna(0, inplace=True)

# 轉換數據類型（確保數值列為數字）
num_cols = df.select_dtypes(include=['object']).columns
for col in num_cols:
    try:
        df[col] = pd.to_numeric(df[col])
    except:
        pass  # 如果轉換失敗，保持原狀
```

---

## **3. 統計分析**
### **1. 年齡與性別分析**
計算不同年齡層和性別的抗憂鬱藥物使用情況：

```python
# 按年齡層計算平均藥物使用人數
age_stats = df.groupby("年齡別")["抗憂鬱藥物使用人數"].mean()

# 按性別計算平均藥物使用人數
gender_stats = df.groupby("性別")["抗憂鬱藥物使用人數"].mean()

print(age_stats)
print(gender_stats)
```

### **2. 地區分析**
計算各縣市的抗憂鬱藥物使用狀況：

```python
region_stats = df.groupby("地區")["抗憂鬱藥物使用人數"].sum().sort_values(ascending=False)

print(region_stats)
```

---

## **4. 製作結果表格**
將統計結果存為 CSV 或 Excel，方便查閱：

```python
# 儲存為 CSV
age_stats.to_csv("age_stats.csv", encoding="utf-8-sig")
gender_stats.to_csv("gender_stats.csv", encoding="utf-8-sig")
region_stats.to_csv("region_stats.csv", encoding="utf-8-sig")

# 儲存為 Excel
with pd.ExcelWriter("analysis_results.xlsx") as writer:
    age_stats.to_frame().to_excel(writer, sheet_name="年齡統計")
    gender_stats.to_frame().to_excel(writer, sheet_name="性別統計")
    region_stats.to_frame().to_excel(writer, sheet_name="地區統計")

print("分析結果已儲存！")
```

---

## **5. 自動化執行**
使用 `schedule` 讓腳本每天定時運行：

```python
import schedule
import time

def job():
    print("開始執行定期更新...")
    csv_file = download_csv(API_URL)
    if csv_file:
        df = pd.read_csv(csv_file, encoding="utf-8")
        
        # 執行資料清理
        df.fillna(0, inplace=True)
        
        # 重新計算統計數據
        age_stats = df.groupby("年齡別")["抗憂鬱藥物使用人數"].mean()
        gender_stats = df.groupby("性別")["抗憂鬱藥物使用人數"].mean()
        region_stats = df.groupby("地區")["抗憂鬱藥物使用人數"].sum().sort_values(ascending=False)
        
        # 儲存結果
        with pd.ExcelWriter("analysis_results.xlsx") as writer:
            age_stats.to_frame().to_excel(writer, sheet_name="年齡統計")
            gender_stats.to_frame().to_excel(writer, sheet_name="性別統計")
            region_stats.to_frame().to_excel(writer, sheet_name="地區統計")

        print("更新完成！")

# 設定每天 10:00 自動執行
schedule.every().day.at("10:00").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)
```

---

## **結論**
這個腳本可以：
1. **定期下載最新的 CSV 資料**
2. **清理與處理數據**
3. **計算年齡、性別、地區的統計數據**
4. **將結果輸出為 Excel 或 CSV**
5. **每天自動運行**

如果你有任何具體需求或需要進一步修改，請讓我知道！🚀